In [1]:
import pandas as pd


In [2]:
df1= pd.read_csv("/Users/marzieh/Desktop/springboard.class/Projects/Capston Two/NYC EH Data Portal - Asthma emergency department visits (adults) (full table).csv")
df2= pd.read_csv("/Users/marzieh/Desktop/springboard.class/Projects/Capston Two/NYC EH Data Portal - Asthma emergency department visits (age 4 and under) (full table).csv")
df3=pd.read_csv("/Users/marzieh/Desktop/springboard.class/Projects/Capston Two/NYC EH Data Portal - Asthma emergency department visits (age 5 to 17) (full table).csv")
df4=pd.read_csv("/Users/marzieh/Desktop/springboard.class/Projects/Capston Two/NYC EH Data Portal - Fine particles (PM 2.5) (full table).csv")
df5=pd.read_csv("/Users/marzieh/Desktop/springboard.class/Projects/Capston Two/NYC EH Data Portal - Nitric oxide (NOx) (full table).csv")
df6=pd.read_csv("/Users/marzieh/Desktop/springboard.class/Projects/Capston Two/NYC EH Data Portal - Nitrogen dioxide (NO2) (full table).csv")
df7=pd.read_csv("/Users/marzieh/Desktop/springboard.class/Projects/Capston Two/NYC EH Data Portal - Ozone (O3) (full table).csv")
df8=pd.read_csv("/Users/marzieh/Desktop/springboard.class/Projects/Capston Two/NYC EH Data Portal - Sulfur dioxide (SO2) (full table).csv")


In [3]:
# finding common column to merge
common_columns= set(df1.columns)& set(df2.columns)&set(df3.columns)&set(df4.columns)&set(df5.columns)&set(df6.columns)&set(df7.columns)&set(df8.columns)
print(common_columns)

{'GeoID', 'Geography', 'TimePeriod', 'GeoType', 'GeoRank'}


In [4]:
print(df1.head())

   TimePeriod GeoType  GeoID  GeoRank                            Geography  \
0        2020      CD    101        6             Financial District (CD1)   
1        2020      CD    102        6     Greenwich Village and Soho (CD2)   
2        2020      CD    103        6  Lower East Side and Chinatown (CD3)   
3        2020      CD    104        6            Clinton and Chelsea (CD4)   
4        2020      CD    105        6                        Midtown (CD5)   

  Age-adjusted rate per 10,000 Estimated annual rate per 10,000 Number  
0                         17.9                             14.6     80  
1                          9.8                              9.7     81  
2                         44.9                             43.9    661  
3                         28.6                             26.6    364  
4                         21.5                             20.0    116  


In [5]:
print(df2.head())

   TimePeriod GeoType  GeoID  GeoRank                            Geography  \
0        2020      CD    101        6             Financial District (CD1)   
1        2020      CD    102        6     Greenwich Village and Soho (CD2)   
2        2020      CD    103        6  Lower East Side and Chinatown (CD3)   
3        2020      CD    104        6            Clinton and Chelsea (CD4)   
4        2020      CD    105        6                        Midtown (CD5)   

  Estimated annual rate per 10,000 Number  
0                            16.6*     7*  
1                             3.2*     1*  
2                             73.0     47  
3                             23.6     11  
4                            32.5*     7*  


In [6]:
# creat a list of our dfs to use in loop

dfs=[df1,df2,df3,df4,df5,df6,df7,df8]

In [7]:
# Initialize merged_df with the first DataFrame
merged_df= dfs[0]

In [ ]:
# Loop through the dataframes and merge them one by one

for df in dfs[1:]:
    merged_df=pd.merge(merged_df,df, on= "Geography", how="outer")

/var/folders/4m/z4mg17k15k1955vh989t499c0000gn/T/ipykernel_15182/4175888946.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'TimePeriod_x', 'GeoID_x', 'GeoType_x', 'GeoRank_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_df=pd.merge(merged_df,df, on= "Geography", how="outer")


In [ ]:
#Loop through the the DataFrames and compare common columns before merging

for df in dfs[1:]:
    # Select only the common columns for comparison
    merged_common=merged_df[common_columns]
    df_common=df[common_columns]
    if not merged_common.equals(df_common):
        raise ValueError("Common columns do not match between DataFrames.")
    else:
         # If they match, perform the merge
        merged_df = pd.merge(merged_df, df, on='Geography', how='outer')   

In [ ]:
# If no error is raised, all common columns match, and I can drop duplicated columns

In [ ]:
print(merged_df.shape)

In [7]:
print(df8.head())

       TimePeriod GeoType  GeoID  GeoRank  \
0  Winter 2015-16      CD    101        6   
1  Winter 2015-16      CD    102        6   
2  Winter 2015-16      CD    103        6   
3  Winter 2015-16      CD    104        6   
4  Winter 2015-16      CD    105        6   

                             Geography  10th percentile ppb  \
0             Financial District (CD1)                  0.2   
1     Greenwich Village and Soho (CD2)                  0.3   
2  Lower East Side and Chinatown (CD3)                  0.2   
3            Clinton and Chelsea (CD4)                  0.2   
4                        Midtown (CD5)                  0.3   

   90th percentile ppb  Mean ppb  
0                  0.2       0.2  
1                  0.5       0.4  
2                  0.4       0.3  
3                  0.4       0.3  
4                  0.5       0.4  
